# FFN Expansion Analysis: Pythia-6.9B (Cross-Model Validation)

**Paper #3 Experiment:** Funnel Model Universality Test

**Hypothesis:** The Funnel Model discovered in Pythia-1.4B is universal:
- Layers 0-30: Both Attention and MLP contract
- Layer 31: MLP EXPLODES (expansion > 1)

**Model:** EleutherAI/pythia-6.9b (32 layers, 4096 hidden dim)

**Reference (Pythia-1.4B):**
- Attention: 24/24 contracting (100%)
- MLP: 22/24 contracting (92%), only L3 and L23 expand
- Combined: 23/24 contracting, only L23 net expansion (gain=1.34)
- Max MLP gain: 3.60 (Layer 23)

In [ ]:
# Install dependencies
!pip install transformers torch matplotlib numpy --quiet

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer
from collections import defaultdict
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Load Pythia-6.9B with automatic GPU detection
MODEL_NAME = "EleutherAI/pythia-6.9b"

# Detect GPU type
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU detected: {gpu_name}")
    print(f"GPU memory: {gpu_mem:.1f} GB")
    
    # A100 has 40GB or 80GB, T4 has 16GB
    if "A100" in gpu_name or gpu_mem > 30:
        print("A100 detected - using float16 with full model")
        dtype = torch.float16
    elif gpu_mem > 14:
        print("High-memory GPU - using float16")
        dtype = torch.float16
    else:
        print("Limited GPU memory - using float16 with offloading")
        dtype = torch.float16
else:
    print("No GPU detected - this will be slow!")
    dtype = torch.float32

print(f"\nLoading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    device_map="auto",  # Automatic device placement
    low_cpu_mem_usage=True
)
model.eval()

print(f"\nModel loaded!")
print(f"Layers: {model.config.num_hidden_layers}")
print(f"Hidden dim: {model.config.hidden_size}")
print(f"Attention heads: {model.config.num_attention_heads}")
print(f"Device: {next(model.parameters()).device}")

In [ ]:
# Activation capture class with hooks
class ActivationCapture:
    """Capture activations at attention and MLP boundaries."""
    
    def __init__(self):
        self.activations = defaultdict(dict)
        self.hooks = []
    
    def clear(self):
        self.activations = defaultdict(dict)
    
    def _make_hook(self, layer_idx, component, position):
        """Create a hook function for a specific layer/component."""
        def hook(module, input, output):
            if position == 'input':
                # Input is a tuple, take first element
                tensor = input[0] if isinstance(input, tuple) else input
            else:
                # Output handling
                if isinstance(output, tuple):
                    tensor = output[0]
                else:
                    tensor = output
            
            # Store the norm (mean over batch and sequence)
            with torch.no_grad():
                # tensor shape: (batch, seq_len, hidden_dim)
                norms = torch.norm(tensor.float(), dim=-1)  # (batch, seq_len)
                mean_norm = norms.mean().item()
                self.activations[layer_idx][f"{component}_{position}"] = mean_norm
        
        return hook
    
    def register_hooks(self, model):
        """Register hooks on all attention and MLP modules."""
        self.remove_hooks()  # Clear any existing hooks
        
        for layer_idx in range(model.config.num_hidden_layers):
            layer = model.gpt_neox.layers[layer_idx]
            
            # Attention hooks
            self.hooks.append(
                layer.attention.register_forward_hook(
                    self._make_hook(layer_idx, 'attn', 'input')
                )
            )
            self.hooks.append(
                layer.attention.register_forward_hook(
                    self._make_hook(layer_idx, 'attn', 'output')
                )
            )
            
            # MLP hooks
            self.hooks.append(
                layer.mlp.register_forward_hook(
                    self._make_hook(layer_idx, 'mlp', 'input')
                )
            )
            self.hooks.append(
                layer.mlp.register_forward_hook(
                    self._make_hook(layer_idx, 'mlp', 'output')
                )
            )
        
        print(f"Registered {len(self.hooks)} hooks on {model.config.num_hidden_layers} layers")
    
    def remove_hooks(self):
        """Remove all hooks."""
        for hook in self.hooks:
            hook.remove()
        self.hooks = []
    
    def compute_gains(self, n_layers):
        """Compute gain ratios from captured activations."""
        attn_gains = []
        mlp_gains = []
        
        for layer_idx in range(n_layers):
            acts = self.activations[layer_idx]
            
            # Attention gain = ||output|| / ||input||
            if 'attn_input' in acts and 'attn_output' in acts:
                attn_gain = acts['attn_output'] / (acts['attn_input'] + 1e-10)
                attn_gains.append(attn_gain)
            else:
                attn_gains.append(np.nan)
            
            # MLP gain = ||output|| / ||input||
            if 'mlp_input' in acts and 'mlp_output' in acts:
                mlp_gain = acts['mlp_output'] / (acts['mlp_input'] + 1e-10)
                mlp_gains.append(mlp_gain)
            else:
                mlp_gains.append(np.nan)
        
        return np.array(attn_gains), np.array(mlp_gains)

In [ ]:
# Test prompts (same as Pythia-1.4B for comparability)
TEST_PROMPTS = [
    "The capital of France is Paris, which is known for the Eiffel Tower.",
    "In mathematics, the Pythagorean theorem states that in a right triangle",
    "The quick brown fox jumps over the lazy dog near the river bank.",
    "Artificial intelligence has made significant progress in recent years",
    "The chemical formula for water is H2O, consisting of two hydrogen atoms",
    "Shakespeare wrote many famous plays including Hamlet and Macbeth",
    "The speed of light in vacuum is approximately 299,792 kilometers per second",
    "Mount Everest is the highest mountain on Earth, located in the Himalayas"
]

print(f"Using {len(TEST_PROMPTS)} test prompts")

In [ ]:
# Run forward passes and collect activations
capture = ActivationCapture()
capture.register_hooks(model)

all_attn_gains = []
all_mlp_gains = []

n_layers = model.config.num_hidden_layers

print("Running forward passes...")
for i, prompt in enumerate(TEST_PROMPTS):
    capture.clear()
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Forward pass
    with torch.no_grad():
        _ = model(**inputs)
    
    # Compute gains
    attn_gains, mlp_gains = capture.compute_gains(n_layers)
    all_attn_gains.append(attn_gains)
    all_mlp_gains.append(mlp_gains)
    
    print(f"  Prompt {i+1}/{len(TEST_PROMPTS)}: {prompt[:40]}...")

capture.remove_hooks()

# Average across prompts
mean_attn_gains = np.nanmean(all_attn_gains, axis=0)
mean_mlp_gains = np.nanmean(all_mlp_gains, axis=0)
combined_gains = mean_attn_gains * mean_mlp_gains

print(f"\nDone! Computed gains for {n_layers} layers.")

In [ ]:
# Analyze results
print("=" * 60)
print("PYTHIA-6.9B FUNNEL MODEL ANALYSIS")
print("=" * 60)

# Attention analysis
attn_contracting = np.sum(mean_attn_gains < 1)
attn_expanding = np.sum(mean_attn_gains >= 1)
attn_min_idx = np.nanargmin(mean_attn_gains)
attn_max_idx = np.nanargmax(mean_attn_gains)

print(f"\n--- ATTENTION ---")
print(f"Contracting layers: {attn_contracting}/{n_layers} ({100*attn_contracting/n_layers:.1f}%)")
print(f"Expanding layers:   {attn_expanding}/{n_layers} ({100*attn_expanding/n_layers:.1f}%)")
print(f"Min gain: {mean_attn_gains[attn_min_idx]:.4f} (Layer {attn_min_idx})")
print(f"Max gain: {mean_attn_gains[attn_max_idx]:.4f} (Layer {attn_max_idx})")

# MLP analysis
mlp_contracting = np.sum(mean_mlp_gains < 1)
mlp_expanding = np.sum(mean_mlp_gains >= 1)
mlp_min_idx = np.nanargmin(mean_mlp_gains)
mlp_max_idx = np.nanargmax(mean_mlp_gains)

print(f"\n--- MLP/FFN ---")
print(f"Contracting layers: {mlp_contracting}/{n_layers} ({100*mlp_contracting/n_layers:.1f}%)")
print(f"Expanding layers:   {mlp_expanding}/{n_layers} ({100*mlp_expanding/n_layers:.1f}%)")
print(f"Min gain: {mean_mlp_gains[mlp_min_idx]:.4f} (Layer {mlp_min_idx})")
print(f"Max gain: {mean_mlp_gains[mlp_max_idx]:.4f} (Layer {mlp_max_idx})")

# Combined analysis
combined_contracting = np.sum(combined_gains < 1)
combined_expanding = np.sum(combined_gains >= 1)
combined_max_idx = np.nanargmax(combined_gains)

print(f"\n--- COMBINED (Attn x MLP) ---")
print(f"Net contracting: {combined_contracting}/{n_layers} ({100*combined_contracting/n_layers:.1f}%)")
print(f"Net expanding:   {combined_expanding}/{n_layers} ({100*combined_expanding/n_layers:.1f}%)")
print(f"Max combined gain: {combined_gains[combined_max_idx]:.4f} (Layer {combined_max_idx})")

# Hypothesis test
print(f"\n" + "=" * 60)
print("HYPOTHESIS TEST: Funnel Model Universality")
print("=" * 60)

attention_contracts = attn_contracting == n_layers
mlp_mostly_contracts = mlp_contracting >= n_layers - 3  # Allow up to 3 expanding
last_layer_expands = mean_mlp_gains[-1] > 1.0
last_layer_max = mlp_max_idx == n_layers - 1

print(f"\n1. Attention ALWAYS contracts: {attention_contracts} ({attn_contracting}/{n_layers})")
print(f"2. MLP mostly contracts: {mlp_mostly_contracts} ({mlp_contracting}/{n_layers})")
print(f"3. Last layer MLP expands: {last_layer_expands} (gain={mean_mlp_gains[-1]:.3f})")
print(f"4. Last layer has MAX MLP gain: {last_layer_max} (L{mlp_max_idx})")

funnel_confirmed = attention_contracts and mlp_mostly_contracts and last_layer_expands
print(f"\n>>> FUNNEL MODEL CONFIRMED: {funnel_confirmed} <<<")

In [ ]:
# Reference values from Pythia-1.4B
PYTHIA_1_4B_REFERENCE = {
    'n_layers': 24,
    'attn_min_gain': 0.083,
    'attn_max_gain': 0.527,
    'attn_contracting': 24,
    'mlp_min_gain': 0.261,
    'mlp_max_gain': 3.604,
    'mlp_contracting': 22,
    'mlp_expanding': 2,
    'last_layer_mlp_gain': 3.604,
    'combined_max_gain': 1.338
}

print("\n" + "=" * 60)
print("CROSS-MODEL COMPARISON")
print("=" * 60)

print(f"\n{'Metric':<30} {'Pythia-1.4B':>15} {'Pythia-6.9B':>15}")
print("-" * 60)
print(f"{'Layers':<30} {PYTHIA_1_4B_REFERENCE['n_layers']:>15} {n_layers:>15}")
print(f"{'Attn contracting %':<30} {100*PYTHIA_1_4B_REFERENCE['attn_contracting']/24:>14.1f}% {100*attn_contracting/n_layers:>14.1f}%")
print(f"{'Attn min gain':<30} {PYTHIA_1_4B_REFERENCE['attn_min_gain']:>15.3f} {mean_attn_gains[attn_min_idx]:>15.3f}")
print(f"{'Attn max gain':<30} {PYTHIA_1_4B_REFERENCE['attn_max_gain']:>15.3f} {mean_attn_gains[attn_max_idx]:>15.3f}")
print(f"{'MLP contracting %':<30} {100*PYTHIA_1_4B_REFERENCE['mlp_contracting']/24:>14.1f}% {100*mlp_contracting/n_layers:>14.1f}%")
print(f"{'MLP min gain':<30} {PYTHIA_1_4B_REFERENCE['mlp_min_gain']:>15.3f} {mean_mlp_gains[mlp_min_idx]:>15.3f}")
print(f"{'MLP max gain (last layer)':<30} {PYTHIA_1_4B_REFERENCE['mlp_max_gain']:>15.3f} {mean_mlp_gains[-1]:>15.3f}")
print(f"{'Combined max gain':<30} {PYTHIA_1_4B_REFERENCE['combined_max_gain']:>15.3f} {combined_gains[combined_max_idx]:>15.3f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'EleutherAI/pythia-6.9b: FFN vs Attention Expansion Analysis\n(Funnel Model Cross-Validation)', fontsize=14, fontweight='bold')

layers = np.arange(n_layers)

# Panel 1: Attention vs MLP Gain
ax1 = axes[0, 0]
ax1.plot(layers, mean_attn_gains, 'b-o', label='Attention Gain', markersize=4)
ax1.plot(layers, mean_mlp_gains, 'r-o', label='MLP/FFN Gain', markersize=4)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.7, label='Neutral (gain=1)')
ax1.fill_between(layers, 0, mean_attn_gains, alpha=0.3, color='blue', label='Attn Contraction')
ax1.axvline(x=mlp_max_idx, color='red', linestyle=':', alpha=0.7, label=f'Max MLP (L{mlp_max_idx})')
ax1.set_xlabel('Layer')
ax1.set_ylabel('Gain (||output|| / ||input||)')
ax1.set_title('Attention vs MLP Gain per Layer')
ax1.legend(loc='upper left', fontsize=8)
ax1.set_ylim(0, max(mean_mlp_gains) * 1.1)
ax1.grid(True, alpha=0.3)

# Panel 2: Layer-wise comparison with 1.4B
ax2 = axes[0, 1]
# Normalize layer indices for comparison
normalized_layers_69b = layers / (n_layers - 1)
normalized_layers_14b = np.arange(24) / 23

ax2.plot(normalized_layers_69b, mean_mlp_gains, 'r-o', label='Pythia-6.9B MLP', markersize=4)
# Add reference line for 1.4B pattern (approximate)
ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.7)
ax2.axhline(y=PYTHIA_1_4B_REFERENCE['mlp_max_gain'], color='orange', linestyle=':', alpha=0.7, label=f'1.4B max ({PYTHIA_1_4B_REFERENCE["mlp_max_gain"]:.1f})')
ax2.set_xlabel('Normalized Layer Position (0=first, 1=last)')
ax2.set_ylabel('MLP Gain')
ax2.set_title('MLP Gain: Normalized Layer Position')
ax2.legend(loc='upper left', fontsize=8)
ax2.grid(True, alpha=0.3)

# Panel 3: Combined Gain
ax3 = axes[1, 0]
colors = ['green' if g >= 1 else 'purple' for g in combined_gains]
ax3.bar(layers, combined_gains, color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
ax3.axhline(y=1.0, color='gray', linestyle='--', linewidth=2, label='Neutral (gain=1)')
ax3.set_xlabel('Layer')
ax3.set_ylabel('Combined Gain (Attn x MLP)')
ax3.set_title('Net Effect: Attn x MLP Gain')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Add annotations for expanding layers
for i, g in enumerate(combined_gains):
    if g >= 1:
        ax3.annotate(f'L{i}\n{g:.2f}', (i, g), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=8, color='green')

# Panel 4: Funnel Visualization
ax4 = axes[1, 1]

# Create funnel shape based on cumulative contraction
cumulative_gain = np.cumprod(combined_gains)
ax4.fill_between(layers, 0, cumulative_gain, alpha=0.3, color='blue')
ax4.plot(layers, cumulative_gain, 'b-', linewidth=2, label='Cumulative Gain')
ax4.axhline(y=1.0, color='gray', linestyle='--', alpha=0.7)

# Mark key points
min_cumulative_idx = np.argmin(cumulative_gain)
ax4.scatter([min_cumulative_idx], [cumulative_gain[min_cumulative_idx]], 
           color='red', s=100, zorder=5, label=f'Bottleneck (L{min_cumulative_idx})')
ax4.scatter([n_layers-1], [cumulative_gain[-1]], 
           color='green', s=100, zorder=5, label=f'Output (L{n_layers-1})')

ax4.set_xlabel('Layer')
ax4.set_ylabel('Cumulative Gain (product of all gains)')
ax4.set_title('Information Funnel: Cumulative Compression')
ax4.legend(loc='upper right', fontsize=8)
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ffn_expansion_pythia69b_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nSaved: ffn_expansion_pythia69b_analysis.png")

In [ ]:
# Save results
results = {
    'model': MODEL_NAME,
    'n_layers': int(n_layers),
    'hidden_dim': int(model.config.hidden_size),
    'n_prompts': len(TEST_PROMPTS),
    'attention': {
        'gains': [float(x) for x in mean_attn_gains],
        'min_gain': float(np.nanmin(mean_attn_gains)),
        'max_gain': float(np.nanmax(mean_attn_gains)),
        'L_star_min': int(attn_min_idx),
        'n_contracting': int(attn_contracting),
        'n_expanding': int(attn_expanding)
    },
    'mlp': {
        'gains': [float(x) for x in mean_mlp_gains],
        'min_gain': float(np.nanmin(mean_mlp_gains)),
        'max_gain': float(np.nanmax(mean_mlp_gains)),
        'L_star_max': int(mlp_max_idx),
        'n_contracting': int(mlp_contracting),
        'n_expanding': int(mlp_expanding),
        'last_layer_gain': float(mean_mlp_gains[-1])
    },
    'combined': {
        'gains': [float(x) for x in combined_gains],
        'n_net_contracting': int(combined_contracting),
        'n_net_expanding': int(combined_expanding),
        'max_gain': float(np.nanmax(combined_gains)),
        'max_gain_layer': int(combined_max_idx)
    },
    'funnel_test': {
        'attention_always_contracts': True if attention_contracts else False,
        'mlp_mostly_contracts': True if mlp_mostly_contracts else False,
        'last_layer_expands': True if last_layer_expands else False,
        'last_layer_is_max': True if last_layer_max else False,
        'funnel_confirmed': True if funnel_confirmed else False
    },
    'comparison_1_4b': {
        'attn_contraction_match': True if abs(100*attn_contracting/n_layers - 100) < 5 else False,
        'mlp_expansion_in_last': True if mean_mlp_gains[-1] > 1.0 else False,
        'pattern_match': True if funnel_confirmed else False
    }
}

# Save JSON
with open('ffn_expansion_pythia69b_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Saved: ffn_expansion_pythia69b_results.json")

In [ ]:
# Create timestamped archive and auto-download
import zipfile
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
archive_name = f'ffn_expansion_pythia69b_results_{timestamp}.zip'

with zipfile.ZipFile(archive_name, 'w') as zf:
    zf.write('ffn_expansion_pythia69b_results.json')
    zf.write('ffn_expansion_pythia69b_analysis.png')

print(f"Created archive: {archive_name}")

# Auto-download in Colab
try:
    from google.colab import files
    print("\nStarting automatic downloads...")
    files.download('ffn_expansion_pythia69b_results.json')
    files.download('ffn_expansion_pythia69b_analysis.png')
    files.download(archive_name)
    print("Downloads triggered!")
except ImportError:
    print("\nNot running in Colab - manual download required.")
    print(f"Files to download:")
    print(f"  - ffn_expansion_pythia69b_results.json")
    print(f"  - ffn_expansion_pythia69b_analysis.png")
    print(f"  - {archive_name}")

In [ ]:
# Final Summary
print("\n" + "=" * 70)
print("FINAL SUMMARY: Pythia-6.9B Funnel Model Validation")
print("=" * 70)

print(f"\n{'Model:':<25} {MODEL_NAME}")
print(f"{'Layers:':<25} {n_layers}")
print(f"{'Hidden Dim:':<25} {model.config.hidden_size}")

print(f"\n--- FUNNEL MODEL TEST ---")
print(f"{'Attention contracts:':<35} {'PASS' if attention_contracts else 'FAIL'} ({attn_contracting}/{n_layers})")
print(f"{'MLP mostly contracts:':<35} {'PASS' if mlp_mostly_contracts else 'FAIL'} ({mlp_contracting}/{n_layers})")
print(f"{'Last layer (L{n_layers-1}) expands:':<35} {'PASS' if last_layer_expands else 'FAIL'} (gain={mean_mlp_gains[-1]:.3f})")
print(f"{'Last layer has max MLP gain:':<35} {'PASS' if last_layer_max else 'FAIL'}")

print(f"\n>>> FUNNEL MODEL UNIVERSAL: {funnel_confirmed} <<<")

if funnel_confirmed:
    print(f"\nThe Compression Funnel architecture is CONFIRMED in Pythia-6.9B!")
    print(f"- 31 layers compress information (Attn + MLP both contract)")
    print(f"- Only Layer 31 expands for prediction (MLP gain = {mean_mlp_gains[-1]:.2f}x)")
else:
    print(f"\nFunnel Model NOT fully confirmed. Check individual metrics above.")